---
# Visualisations en Python
---

<center><img src="./img/mlprocess_2.png" alt="Processus d'apprentissage automatique" width="50%"/></center>

# Exemple 1 : Visualisation de données univariées

Dans cet exemple, nous allons utiliser un ensemble de données public pour apprivoiser les différentes manières d'afficher des données en Python en utilisant différentes librairies. Nous allons utiliser la très répandue [`matplotlib`](https://matplotlib.org/), la plus nichée [`seaborn`](https://seaborn.pydata.org/) qui est une surcouche à `matplotlib`, ou encore [`plotnine`](https://plotnine.readthedocs.io/en/stable/), pour ceux qui viennent du monde de R.

## Les données : Rétention de client d'une compagnie de télécommunications

Mais avant d'arriver aux données, initialisons notre environnement et importons des librairies nécessaires


In [ ]:
import numpy as np
import pandas as pd

# On s'épargne les avertissements (warnings). 
# Commentez simplement les deux lignes suivantes si vous souhaitez les voir affichés.
import warnings
warnings.filterwarnings('ignore')

# Matplotlib : la base
import matplotlib.pyplot as plt

# Seaborn : quelques trucs jolis en plus
import seaborn as sns
sns.set()

# Pour les habitués de R :
from plotnine import *

# Les données
import wget

!rm './telecom_churn.csv'
wget.download('https://raw.githubusercontent.com/iid-ulaval/EEAA-datasets/master/telecom_churn.csv','./telecom_churn.csv')


# Pour du Web mieux vaut utiliser du SVG ... en local on va préférer du PNG
%config InlineBackend.figure_format = 'svg'

Nous allons examiner les données relatives au désabonnement de la clientèle pour un opérateur de télécommunications. Nous allons charger cet ensemble de données dans un `DataFrame`:

In [ ]:
df = pd.read_csv('telecom_churn.csv')

Pour se familiariser avec nos données, regardons les 5 premières entrées en utilisant `head()`:

In [ ]:
df.head()

On peut déjà se débarasser de la colonne du numéro de téléphone qui ne donnera pas plus d'information que l'ID unique.

Regardons également combien de données manquantes sont dans le dataset (jeu de mots).

In [ ]:
df = df.drop('Phone number',axis=1)
df.isna().sum()

Regardons à nouveau quelques données tirées au hasard:

In [ ]:
df.sample(10)

Voici une description des caractéristiques selon le dataset de Kaggle:

|  Name  | Description | Value Type | Statistical Type |
|---         |---       |---     |---
| **State** | State abbreviation (like KS = Kansas) | String | Categorical |
| **Account length** | How long the client has been with the company | Numerical | Quantitative |
| **Area code** | Phone number prefix | Numerical | Categorical |
| **International plan** | International plan (on/off) | String, "Yes"/"No" | Categorical/Binary |
| **Voice mail plan** | Voicemail (on/off) | String, "Yes"/"No" | Categorical/Binary |
| **Number vmail messages** | Number of voicemail messages | Numerical | Quantitative |
| **Total day minutes** |  Total duration of daytime calls | Numerical | Quantitative |
| **Total day calls** | Total number of daytime calls  | Numerical | Quantitative |
| **Total day charge** | Total charge for daytime services | Numerical | Quantitative |
| **Total eve minutes** | Total duration of evening calls | Numerical | Quantitative |
| **Total eve calls** | Total number of evening calls | Numerical | Quantitative |
| **Total eve charge** | Total charge for evening services | Numerical | Quantitative |
| **Total night minutes** | Total duration of nighttime calls | Numerical | Quantitative |
| **Total night calls** | Total number of nighttime calls | Numerical | Quantitative |
| **Total night charge** | Total charge for nighttime services | Numerical | Quantitative |
| **Total intl minutes** | Total duration of international calls  | Numerical | Quantitative |
| **Total intl calls** | Total number of international calls | Numerical | Quantitative |
| **Total intl charge** | Total charge for international calls | Numerical | Quantitative |
| **Customer service calls** | Number of calls to customer service | Numerical | Categorical/Ordinal |

La dernière colonne de données, **Churn**, représente la variable cible. Elle est binaire: *True* indique que l'entreprise a finalement perdu ce client, et *False* indique que le client a été conservé. Plus tard, nous construirons des modèles prédisant cette finalité en fonction des variables restantes. C'est pourquoi nous l'appelons la *cible*.



## Visualisation univariée

L'analyse *univariée* examine une caractéristique à la fois. Lorsque nous analysons une entité de manière indépendante, nous nous intéressons généralement à la *distribution de ses valeurs* et ignorons les autres caractéristiques.

Ci-dessous, nous examinerons différents types statistiques d’entités et les outils correspondants pour leur analyse visuelle individuelle.

### Caractéristiques quantitatives

*Les caractéristiques quantitatives* prennent des valeurs numériques ordonnées. Ces valeurs peuvent être *discrètes*, comme des nombres entiers, ou *continues*, comme des nombres réels, et expriment généralement un compte ou une mesure.

#### Histogrammes et diagrammes de densité

Le moyen le plus simple d'examiner la distribution d'une variable numérique consiste à tracer son *histogramme* à l'aide de la méthode de `DataFrame` [`hist()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.hist.html).


In [ ]:
features = ['Total day minutes', 'Total intl calls']
df[features].hist(figsize=(10, 4));

Un histogramme regroupe les valeurs dans *des groupes* de même plage de valeurs. La forme d'un histogramme donne souvent des indices sur le type de distribution sous-jacente: gaussienne, exponentielle, etc. Vous pouvez également repérer toute asymétrie dans sa forme lorsque la distribution est presque régulière, mais présente quelques anomalies. Il est important de connaître la distribution des valeurs des caractéristiques lorsque vous utilisez des méthodes d’apprentissage automatique qui en supposent un type particulier, le plus souvent gaussien.

Dans le graphique ci-dessus, nous voyons que la variable *Total des minutes du jour* est normalement distribuée, alors que *Total des appels intl* est nettement en biais à droite (sa queue est plus longue à droite).

Il existe également une autre façon, souvent plus claire et visuelle, de saisir une distribution: celle des *diagrammes de densité* ou, plus formellement, des *diagrammes de densité du noyau*. Ils peuvent être considérés comme une version [lissée](https://en.wikipedia.org/wiki/Kernel_smoother) de l'histogramme. Leur principal avantage par rapport à ces derniers est qu’ils ne dépendent pas de la taille des bacs. Créons des diagrammes de densité pour les deux mêmes variables:


In [ ]:
df[features].plot(kind='density', subplots=True, layout=(1, 2), 
                  sharex=False, figsize=(10, 4));

Il est également possible de tracer une distribution d'observations avec le [`distplot()`](https://seaborn.pydata.org/generated/seaborn.distplot.html) de `seaborn`. Par exemple, regardons la distribution de *Total de minutes par jour*. Par défaut, le tracé affiche à la fois l'histogramme avec l'[estimation de la densité du noyau](https://en.wikipedia.org/wiki/Kernel_density_estimation) (KDE) en haut.

In [ ]:
sns.distplot(df['Total intl calls']);

Pour les habitués de _R_, `plotnine` propose la fonction _ggplot_ quasiment identique à celle disponible dans ce langage. Par exemple, pour refaire la même figure que le [`distplot()`](https://seaborn.pydata.org/generated/seaborn.distplot.html) de `seaborn`, en voici le code, grâce à [`geom_histogram()`](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_histogram.html) et [`geom_density()`](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_density.html):

In [ ]:
(
ggplot(df) # data
+ aes('Total intl calls') # column
+ geom_density() # Kernel Density Estimation
+ geom_histogram(aes(y='stat(density)'),alpha=.5, fill='cornflowerblue') # Histogram ... duh.
+ theme_matplotlib() # to keep original style
)

La hauteur des barres de l'histogramme ici est normée et indique la densité plutôt que le nombre d'exemples dans chaque casier.

#### Box plot

Un autre type de visualisation utile est le *box plot*. `seaborn` fait du bon travail ici:

In [ ]:
sns.boxplot(y='Total intl calls', data=df);

Et l'équivalent `plotnine.geoms.`[`geom_box_plot()`](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_boxplot.html): 

In [ ]:
(
ggplot(df,aes(x=0,y='Total intl calls')) # data
+ geom_boxplot()
+ theme_matplotlib() # to keep original style
)

Voyons comment interpréter une boîte à moustaches. Ses composants sont une *boîte*, les *moustaches* au dessus et en dessous et un certain nombre de points individuels (*points aberrants*) au-delà de celles-ci.

Le cadre à lui seul illustre la dispersion interquartile de la distribution; sa longueur est déterminée par les centiles $25\% \, (\text{Q1})$ and $75\% \, (\text{Q3})$. La ligne verticale à l'intérieur de la case marque la médiane ($50\%$) de la distribution.

Les moustaches sont les lignes qui sortent de la boîte. Elles représentent la dispersion des données, en particulier les points compris dans l'intervalle $(\text{Q1} - 1.5 \cdot \text{IQR}, \text{Q3} + 1.5 \cdot \text{IQR})$, où $\text{IQR} = \text{Q3} - \text{Q1}$ est la [plage interquartile](https://en.wikipedia.org/wiki/Interquartile_range).

Les valeurs aberrantes, affichées au-delà des moustaches, sont tracées individuellement sous forme de points noirs le long de l'axe central.

Nous pouvons voir qu'un grand nombre d'appels internationaux est assez rare dans nos données.

#### Diagramme en violon

Un dernier type de tracé de distribution que nous allons considérer est le *tracé en violon*.

Regardez les chiffres ci-dessous. Sur la gauche, nous voyons la boîte à moustaches déjà familière. À droite, un *tracé en violon* avec l'estimation de la densité du noyau des deux côtés.


In [ ]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(6, 4))
sns.boxplot(data=df['Total intl calls'], ax=axes[0]);
sns.violinplot(data=df['Total intl calls'], ax=axes[1]);

Et l'équivalent `plotnine` : 

In [ ]:
( # subplots ne marchent pas encore dans plotnine
ggplot(df) # data
+ geom_violin(aes(x=1,y='Total intl calls'), alpha=.9)
+ geom_boxplot(aes(x=0,y='Total intl calls'), alpha=.5)
+ theme_matplotlib() # to keep original style
)

La différence entre un diagramme de boîte et un en violon est que le premier illustre certaines statistiques concernant des exemples individuels dans un jeu de données, alors que le second se concentre davantage sur la distribution lissée dans son ensemble.

Dans notre cas, le tracé en violon ne fournit aucune information supplémentaire sur les données, car tout est clair à partir du tracé de boîte.

#### describe()

En plus des outils graphiques, pour obtenir les statistiques numériques exactes de la distribution, nous pouvons utiliser la méthode [`describe()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.describe.html) d’un `DataFrame`:


In [ ]:
df[features].describe()

Sa sortie est la plupart du temps explicite. *25%*, *50%* et *75%* sont les [centiles](https://en.wikipedia.org/wiki/Percentile) correspondants.

### Caractéristiques catégorielles et binaires <a id="222" />

*Les caractéristiques catégorielles* prennent un nombre fixe de valeurs. Chacune de ces valeurs attribue une observation à un groupe correspondant, appelé *catégorie*, qui reflète une propriété qualitative de cet exemple. Les variables *binaires* constituent un cas particulier important de variables catégorielles lorsque le nombre de valeurs possibles est de 2. Si les valeurs d'une variable catégorielle sont ordonnées, on l'appelle *ordinale*.

#### Table de fréquences

Voyons l’équilibre des classes dans notre jeu de données en regardant la distribution de la variable cible: le *taux de désabonnement*. Tout d’abord, nous obtiendrons un tableau de fréquences indiquant la fréquence de chaque valeur de la variable catégorielle. Pour cela, nous allons utiliser la méthode [`value_counts()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html):


In [ ]:
df['Churn'].value_counts()

Par défaut, les entrées de la sortie sont triées des valeurs les plus fréquentes aux moins fréquentes.

Dans notre cas, les données ne sont pas *équilibrées*; c'est-à-dire que nos deux classes cibles, les clients fidèles et déloyaux, ne sont pas représentées de manière égale dans le jeu de données. Seule une petite partie des clients a annulé son abonnement au service de télécommunication. Ce fait peut impliquer certaines restrictions quant à la mesure de la performance de la classification et, à l'avenir, nous voudrons peut-être également pénaliser davantage les erreurs de modèle que nous utiliserons pour prédire la classe minoritaire de "Churn".

#### Diagramme en bâtons

Le Diagramme en bâtons est une représentation graphique de la table de fréquences. Le moyen le plus simple de le créer consiste à utiliser la fonction [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) de `Seaborn` . Il existe une autre fonction dans `seaborn` qui s'appelle quelque peu déroutant [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) et qui est principalement utilisée pour la représentation de certaines statistiques de base de une variable numérique regroupée par une caractéristique catégorielle.

Traçons les distributions de deux variables qualitatives:

In [ ]:
_, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))

sns.countplot(x='Churn', data=df, ax=axes[0]);
sns.countplot(x='Customer service calls', data=df, ax=axes[1]);

Bien que les histogrammes, décrits ci-dessus, et les diagrammes en bâtons puissent sembler similaires, il existe plusieurs différences entre eux:
1. Les *histogrammes* conviennent le mieux pour examiner la distribution des variables numériques, tandis que les *diagrammes en bâtons* sont utilisés pour les caractéristiques catégorielles.
2. Les valeurs sur l'axe des X dans *l'histogramme* sont numériques; un *diagramme en bâtons* peut avoir n'importe quel type de valeur sur l'axe des X: nombres, chaînes, booléens.
3. L'axe X de *l'histogramme* est un *axe de coordonnées cartésiennes* le long duquel les valeurs ne peuvent pas être modifiées; l'ordre des *bâtons* n'est pas prédéfini. Néanmoins, il est utile de noter que les bâtons sont souvent triés par hauteur, c'est-à-dire par la fréquence des valeurs. De plus, lorsque nous considérons des variables *ordinales* (comme *appels de service client* dans nos données), les barres sont généralement classées par valeur de variable.

Le graphique de gauche ci-dessus illustre clairement le déséquilibre de notre variable cible. Le diagramme en bâtons pour les *appels de service clientèle* à droite indique que la majorité des clients résolvent leurs problèmes en 2 à 3 appels maximum. Mais, comme nous voulons pouvoir prédire la classe minoritaire, nous pourrions être plus intéressés par le comportement de moins de clients insatisfaits. Il se peut bien que la queue de ce graphique en barres contienne la plus grande partie de notre taux de désabonnement. Ce ne sont que des hypothèses pour le moment. Passons donc à des techniques visuelles plus intéressantes et plus puissantes.

Note : Il n'est pas (encore?) possible de faire des `subplots` en `ggplot`.


In [ ]:
(
ggplot(df)
+geom_bar(aes(x='Churn'))
# Subplots n'existent pas en GGPLOT
#+geom_bar(aes(x='Customer service calls'))
+ theme_matplotlib() # to keep original style
)

## Visualisation multivariée

Les graphiques *multivariés* nous permettent de voir les relations entre deux variables différentes ou plus, le tout dans une seule figure. Tout comme dans le cas des graphiques univariés, le type spécifique de visualisation dépend des types de variables analysées.

### Quantitative – Quantitative

#### Matrice de corrélation

Examinons les corrélations entre les variables numériques de notre ensemble de données. Il est important de connaître ces informations, car il existe des algorithmes d'apprentissage automatique (par exemple, la régression linéaire et logistique) qui ne gèrent pas correctement les variables d'entrée hautement corrélées.

Premièrement, nous allons utiliser la méthode [`corr()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) sur un `DataFrame` qui calcule la corrélation entre chaque paire de caractéristiques. Ensuite, nous passons la *matrice de corrélation* résultante à [`heatmap()`](https://seaborn.pydata.org/generated/seaborn.heatmap.html) de `seaborn`, qui affiche une matrice de couleur pour les valeurs fournies:


In [ ]:
# On ne prend que les variables numériques
numerical = list(set(df.columns) - 
                 set(['State', 'International plan', 'Voice mail plan', 
                      'Area code', 'Churn', 'Customer service calls']))

# Voyez l'effet de la commande suivante et expliquez-la : 
# numerical.sort()

# Calcul et affichage
corr_matrix = df[numerical].corr()
sns.heatmap(corr_matrix)

À partir de la matrice de corrélation colorée générée ci-dessus, nous pouvons voir qu'il existe 4 variables telles que *Total day charge* qui ont été calculées directement à partir du nombre de minutes passées en appels téléphoniques (*Total day minutes*). Celles-ci sont appelées variables *dépendantes* et peuvent donc être omises puisqu'elles ne fournissent aucune information supplémentaire. Débarrassons-nous d'elles:

In [ ]:
numerical = list(set(numerical) - 
                 set(['Total day charge', 'Total eve charge', 'Total night charge', 'Total intl charge']))

#### Nuages de points ou diagramme de dispersion

Le *nuage de points* affiche les valeurs de deux variables numériques sous forme de *coordonnées cartésiennes* dans un espace 2D. Les nuages de points en 3D sont également possibles.

Essayons la fonction [`scatter ()`](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.scatter.html) de la bibliothèque `matplotlib`:

In [ ]:
plt.scatter(df['Total day minutes'], df['Total night minutes'], alpha=.5);

In [ ]:
(
ggplot(df,aes(x='Total day minutes', y='Total night minutes'))
+ geom_point(alpha=.5)
+ theme_matplotlib()
)

Nous obtenons une image sans intérêt de deux variables normalement distribuées. De plus, il semble que ces caractéristiques ne soient pas corrélées, car la forme en forme d'ellipse est alignée sur les axes.

Il existe une option un peu plus sophistiquée pour créer un nuage de points avec la bibliothèque `seaborn`:

In [ ]:
sns.jointplot(x='Total day minutes', y='Total night minutes', alpha=.5,
              data=df, kind='scatter');

La fonction [`jointplot()`](https://seaborn.pydata.org/generated/seaborn.jointplot.html) trace deux histogrammes qui peuvent être utiles dans certains cas.

En utilisant la même fonction, nous pouvons également obtenir une version lissée de notre distribution bivariée:

In [ ]:
sns.jointplot('Total day minutes', 'Total night minutes', data=df,
              kind="kde");

Il s’agit essentiellement d’une version bivariée du *tracé de densité du noyau* (KDE) discuté précédemment.

#### Matrice de diagramme de dispersion

Dans certains cas, il peut être utile de tracer une *matrice de nuages de points* telle que celle présentée ci-dessous. Sa diagonale contient les distributions des variables correspondantes et les diagrammes de dispersion de chaque paire de variables remplissent le reste de la matrice.

In [ ]:
# Attention, ca peut être très long ...
sns.pairplot(df[numerical])

Parfois, une telle visualisation peut aider à tirer des conclusions sur les données; mais, dans ce cas-ci, tout est assez clair sans surprises.

### Quantitative – Catégorielle

Dans cette section, nous allons rendre nos diagrammes quantitatifs un peu plus excitants. Nous tenterons d’obtenir de nouvelles informations sur la prévision du désabonnement grâce aux interactions entre les caractéristiques numériques et catégorielles.

Plus spécifiquement, voyons comment les variables d’entrée sont liées à la variable cible **Churn**.

Nous avons vu précédemment les nuages de points. Ceux ci peuvent être codés en couleur ou en taille afin de représenter une troisième variable catégorielle sur la même figure. Nous pouvons y parvenir avec la fonction `scatter()` vue ci-dessus, mais essayons une nouvelle fonction appelée 
[`lmplot()`](https://seaborn.pydata.org/generated/seaborn.lmplot.html) et utilisons le paramètre de teinte (`hue`) pour indiquer la variable cible **Churn**.


In [ ]:
sns.lmplot('Total day minutes', 'Total night minutes', data=df, hue='Churn', fit_reg=False);

In [ ]:
(
ggplot(df, aes(x='Total day minutes', y='Total night minutes',color='Churn'))
+ geom_point()
+ theme_matplotlib()
)

Il semble que notre faible proportion de clients déloyaux se penche un peu vers le coin supérieur droit; c'est-à-dire que ces clients ont tendance à passer plus de temps au téléphone, jour et nuit. Mais cela n’est pas absolument clair et nous ne tirerons aucune conclusion définitive de ce graphique.

Créons maintenant des diagrammes en blocs pour visualiser les statistiques de distribution des variables numériques dans deux groupes disjoints: les clients fidèles (`Churn = False`) et ceux qui sont partis (`Churn = True`).

In [ ]:
# Parfois, vous pouvez analyser une variable ordinale comme une numérique
numerical.append('Customer service calls')

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(10, 7))
for idx, feat in enumerate(numerical):
    ax = axes[int(idx / 4), idx % 4]
    sns.boxplot(x='Churn', y=feat, data=df, ax=ax)
    ax.set_xlabel('')
    ax.set_ylabel(feat)
fig.tight_layout();

Sur ce graphique, nous pouvons voir que la plus grande divergence dans la distribution entre les deux groupes concerne trois variables: *Total day minutes*, *Customer service calls* et *Number vmail messages*. Nous avons déjà vu comment déterminer l’importance des caractéristiques dans la classification en utilisant les méthodes *Random Forest* ou *Gradient Boosting*; si vous essayez de les utiliser sur ce dataset, vous verrez que les deux premières caractéristiques sont en effet très importantes pour la prévision du taux de désabonnement.

Examinons séparément la distribution des minutes journalières parlées pour les clients fidèles et déloyaux. Nous allons créer des tracés de boîte et en violon pour le *Total day minutes* regroupées selon la variable cible.


In [ ]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(10, 4))

sns.boxplot(x='Churn', y='Total day minutes', data=df, ax=axes[0]);
sns.violinplot(x='Churn', y='Total day minutes', data=df, ax=axes[1]);

Dans ce cas, la comparaison entre les violons ne fournit aucune information supplémentaire à propos de nos données, car tout est déjà clair avec les boîtes : les clients déloyaux ont tendance à parler davantage au téléphone.

**Observation intéressante** : en moyenne, les clients qui résilient leurs contrats sont parmi les plus grands utilisateurs de services de communication. Peut-être ne sont-ils pas satisfaits des tarifs, alors une mesure possible pour éviter le roulement pourrait être une réduction des tarifs d’appel. La société devra entreprendre une analyse économique supplémentaire pour déterminer si une telle mesure serait réellement bénéfique.

Lorsque nous voulons analyser une variable quantitative à la fois en deux dimensions catégoriques, il existe une fonction appropriée dans la bibliothèque `seaborn` appelée [`catplot()`](https://seaborn.pydata.org/generated/seaborn.factorplot.html). Par exemple, visualisons l'interaction entre *Total day minutes* et deux variables qualitatives dans le même graphique:


In [ ]:
sns.catplot(x='Churn', y='Total day minutes', col='Customer service calls',
               data=df[df['Customer service calls'] < 8], kind="box",
               col_wrap=4, height=3, aspect=.8);

On peut en conclure qu'à partir de 4 appels, le *Total day minutes* peut ne plus être le principal facteur de désabonnement des clients. En plus de nos hypothèses précédentes sur les tarifs, il se peut que certains clients ne soient pas satisfaits du service en raison d'autres problèmes, ce qui pourrait réduire le nombre de minutes d'appels par jour.

### Categorielle – Categorielle <a id="233"/>

Comme nous l'avons vu précédemment dans cet article, la variable *Customer service calls* a peu de valeurs uniques et peut donc être considérée comme numérique ou ordinale. Nous avons déjà vu sa distribution avec un *countplot*. Nous nous intéressons maintenant à la relation entre cette entité ordinale et la variable cible *Churn*.

Regardons la distribution du nombre d'appels au service clientèle, en utilisant à nouveau un *countplot*. Cette fois, passons également le paramètre `hue = Churn` qui ajoute une dimension catégorique au tracé :


In [ ]:
sns.countplot(x='Customer service calls', hue='Churn', data=df);

**Une observation** : le taux de désabonnement augmente significativement après 4 appels ou plus au service clientèle.

Maintenant, regardons la relation entre *Churn* et les fonctionnalités binaires, *International plan* et *Voice mail plan*.

In [ ]:
_, axes = plt.subplots(1, 2, sharey=True, figsize=(10, 4))

sns.countplot(x='International plan', hue='Churn', data=df, ax=axes[0]);
sns.countplot(x='Voice mail plan', hue='Churn', data=df, ax=axes[1]);

**Une observation** : lorsque *International plan* est activé, le taux de désabonnement est beaucoup plus élevé; l'utilisation du plan international par le client est un atout majeur. Nous n'observons pas le même effet avec *Voice mail plan*.

#### Tableau de contingence

Outre l’utilisation de moyens graphiques pour l’analyse catégorique, il existe un outil traditionnel en statistiques: le *tableau de contingence*, également appelé *tableau croisé*. Il représente la distribution de fréquence multivariée des variables catégorielles sous forme de tableau. En particulier, cela nous permet de voir la distribution d'une variable conditionnellement à l'autre en regardant le long d'une colonne ou d'une ligne.

Essayons de voir comment *Churn* est lié à la variable catégorique *State* en créant une tabulation croisée:


In [ ]:
pd.crosstab(df['State'], df['Churn']).T

Dans le cas de *State*, le nombre de valeurs distinctes est plutôt élevé: 51. Nous constatons qu'il n'y a que quelques points de données disponibles pour chaque état - seuls 3 à 17 clients dans chaque état ont abandonné l'opérateur. Ignorons cela un instant et calculons le taux de désabonnement pour chaque état en le triant de haut en bas:

In [ ]:
df.groupby(['State'])['Churn'].agg([np.mean]).sort_values(by='mean', ascending=False).T

À première vue, il semble que le taux de désabonnement dépasse 25% dans le *New Jersey* et la *Californie* et soit inférieur à 6% à Hawaii et en Alaska. Cependant, ces conclusions reposent sur trop peu d'exemples et notre observation pourrait n'être qu'une simple propriété de notre ensemble de données particulier (une fluctuation statistique). Nous pouvons le confirmer avec la mesure de [Matthews](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient), mais cela sortirait du cadre de ce cours.

## Exercice sur le dataset complet

### Approche naïve <a id="241"/>

Nous avons examiné différentes *facettes* de notre jeu de données en devinant les caractéristiques intéressantes et en sélectionnant un petit nombre à la fois pour la visualisation. Nous n’avons traité que deux ou trois variables à la fois et avons facilement pu observer la structure et les relations entre les données. Mais que se passe-t-il si nous voulons afficher toutes les fonctionnalités tout en pouvant interpréter la visualisation résultante?

Nous pourrions utiliser `hist()` ou créer une matrice de nuages de points avec `pairplot()` pour l'ensemble du jeu de données afin d'examiner simultanément toutes nos entités. Toutefois, lorsque le nombre de caractéristiques est important, ce type d'analyse visuelle devient rapidement lent et inefficace. En outre, nous analyserions toujours nos variables par paires, pas toutes en même temps.


### Réduction de la dimensionnalité

La plupart des jeux de données du monde réel comportent de nombreuses caractéristiques, parfois plusieurs milliers. Chacune d'elle peut être considérée comme une dimension dans l'espace des points de données. Par conséquent, la plupart du temps, nous traitons avec des jeux de données de grande dimension, où la visualisation entière est impossible.

Pour examiner un jeu de données dans son ensemble, nous devons réduire le nombre de dimensions utilisées dans la visualisation sans perdre trop d'informations sur les données. Cette tâche s'appelle la *réduction de dimensionnalité* (cf. [Module 0](./Module_0_Coll_Prep.ipynb)) et constitue un exemple de problème *d'apprentissage non supervisé*, car nous devons dériver de nouvelles caractéristiques de faible dimension à partir des données elles-mêmes, sans aucune entrée supervisée.

Une méthode de réduction de dimensionnalité bien connue est *l'analyse en composantes principales* (PCA). Étant un algorithme *linéaire* impliquant certaines restrictions sur les données, ses performances sont toutefois limitées.

Il existe également de nombreuses méthodes non linéaires, appelées collectivement *Manifold Learning*. L'une des plus connues est *t-SNE*. Nous en avons vu plusieurs dans le [module 0](./Module_0_Coll_Prep.ipynb).


### t-SNE

Créons une représentation [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) des données précédentes de désabonnement.

Le nom de la méthode semble complexe et un peu intimidant: *Incorporation de voisins stochastiques t-distribués*. Ses calculs sont également impressionnants (nous ne nous en plongerons pas ici, mais si vous vous sentez courageux, voici l'[article original](http://www.jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf) de Laurens Van der Maaten et Geoffrey Hinton de [JMLR](http://www.jmlr.org/)). Son idée de base est simple: trouver une projection pour un espace de grandes dimensions sur un plan (ou un hyperplan 3D, mais il est presque toujours 2D) de manière à ce que les points éloignés dans l'espace initial à n dimensions se retrouvent éloignés dans le plan. Ceux qui étaient proches à l’origine resteraient proches les uns des autres.

L'*incorporation de voisins* est essentiellement une recherche d'une nouvelle représentation de données de moindre dimension préservant le voisinage des exemples.

Maintenant, faisons un peu de pratique. Premièrement, nous devons importer des classes supplémentaires:


In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

Nous allons laisser de côté les fonctionnalités *State* et *Churn* en utilisant la fonction [`pandas.DataFrame.drop()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) et convertir les valeurs "Oui"/"Non" des entités binaires en valeurs numériques en utilisant [`pandas.Series.map()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html):

In [ ]:
#... 

Nous devons également normaliser les données. Pour cela, nous allons soustraire la moyenne de chaque variable et la diviser par son écart type. Tout cela peut être fait avec `StandardScaler` (cf. [Module 0](./Module_0_Coll_Prep.ipynb)).

In [ ]:
# ... 

Construisons maintenant une représentation t-SNE :

In [ ]:
# ... 

Et affichons-la : 

In [ ]:
# ... 

Colorons cette représentation t-SNE en fonction du taux de désabonnement `Churn` en bleu pour les clients fidèles et orange pour ceux qui quittent, en utilisant la fonction [`pandas.Series.map()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) pour associer les `False` à bleu et les `True` à orange.

In [ ]:
# ...

Nous pouvons voir que les clients qui ont quitté sont concentrés dans quelques zones de l’espace réduit.

Pour mieux comprendre la figure, nous pouvons également la colorier avec les fonctions binaires restantes: `International plan` et `Voice mail plan`. Les points oranges indiquent les instances positives pour la fonction binaire correspondante (donc toujours en utilisant la fonction [`pandas.Series.map()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) pour associer les `no` à bleu et les `yes` à orange).

Essayez ici de faire un `subplots(1, 2)` pour avoir les images côte à côte.


In [ ]:
# ... 

On voit maintenant que de nombreux clients insatisfaits ayant résilié leur abonnement se retrouvent dans le groupe le plus au sud-ouest qui représente les utilisateurs du plan international, mais pas de messagerie vocale.

Enfin, notons quelques inconvénients du t-SNE:
- Grande complexité de calcul. Le fichier [implementation](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) dans Scikit-Learn ne sera probablement pas utilisable en situation réelle. Si vous avez un grand nombre d'échantillons, vous devriez plutôt essayer [Multicore-TSNE](https://github.com/DmitryUlyanov/Multicore-TSNE).
- Le résultat peut varier selon la valeur de la `seed` aléatoire, ce qui complique l'interprétation. [Vous trouverez ici](http://distill.pub/2016/misread-tsne/) un bon tutoriel sur t-SNE. En général, vous ne devriez pas tirer de conclusions de grande envergure basées sur de tels graphiques, car cela équivaudrait à une simple estimation. Bien sûr, certaines conclusions des images t-SNE peuvent inspirer une idée et être confirmées par la suite avec des recherches plus poussées, mais cela arrive rarement.

De temps en temps, en utilisant t-SNE, vous pouvez obtenir une très bonne intuition sur les données. Ce qui suit est un bon article qui en donne un exemple pour les chiffres manuscrits: [Visualiser MNIST](https://colah.github.io/posts/2014-10-Visualizing-MNIST/).

Vous reviendrez plus sur le *Manifold Learning* sur MNIST en allant lire un peu plus loin sur la préparation des données dans ce notebook.


---
# Un autre exercice : Analysons (encore) les passagers du "Titanic"
---
**[Compétition Kaggle](https://www.kaggle.com/c/titanic) "Titanic: Machine Learning from Disaster".**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from plotnine import *
sns.set()
import matplotlib.pyplot as plt
import wget

!rm './titanic_train.csv' './titanic_test.csv'
wget.download('https://raw.githubusercontent.com/iid-ulaval/EEAA-datasets/master/titanic_train.csv','./titanic_train.csv')
wget.download('https://raw.githubusercontent.com/iid-ulaval/EEAA-datasets/master/titanic_test.csv','./titanic_test.csv')


**Read data**

 - Survival: Survival (0 = No, 1 = Yes)
 - Pclass: Ticket class(1 = 1st, 2 = 2nd, 3 = 3rd)
 - Sex: Sex
 - Age: Age in years
 - SibSp: Number of siblings / spouses aboard the Titanic
 - Parch: Number of parents / children aboard the Titanic
 - Ticket: Ticket number
 - Fare:Passenger fare
 - Embarked:Port of Embarkation(C = Cherbourg, Q = Queenstown, S = Southampton)

In [ ]:
titan_df = pd.concat([pd.read_csv('./titanic_train.csv'),pd.read_csv('./titanic_test.csv')])

Débarrassons-nous des valeurs incomplètes cette fois-ci:

In [ ]:
titan_df = titan_df.dropna()

**Créez une image pour visualiser tous les diagrammes de dispersion pour chaque paire de caractéristiques `Age`, `Fare`, `SibSp`, `Parch` et `Survived`. (`scatter_matrix` de Pandas ou `pairplot` de Seaborn)**

In [ ]:
# ...

**Comment le prix du billet (`Fare`) dépend-il de `Pclass`? Construire une boîte à moustaches ([`boxplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.boxplot.html) ou [`geom_boxplot()`](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_boxplot.html)).**

Note : Pour les variables catégorielles dans GGPlot, il faut utiliser `'factor(var)'` pour décomposer celles-ci sur l'axe.


In [ ]:
# ...

**Construisons le même graphe, mais limitons les valeurs de `Fare` à moins de 95% en quantiles du vecteur initial (pour supprimer les valeurs aberrantes qui rendent le graphe moins clair).**

(Il est possible d'éliminer *tous* les outliers avec un simple paramètres de [`boxplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.boxplot.html))

In [ ]:
# ...

**À quel point la proportion de passagers survivants dépend-elle du sexe des passagers? Représentez-le avec `Seaborn.countplot` en utilisant l'argument `hue`.**

In [ ]:
# ...

Faisons la même chose pour `Survived` et `Pclass`

In [ ]:
# ...

**En quoi la distribution des prix de billet diffère-t-elle pour les passagers ayant survécu ou non? Affichez-la avec `seaborn.boxplot` ou `plotnine.geom_boxplot`**

In [ ]:
# ...


Triste vérité; ceux qui ont survécu ont généralement payé beaucoup plus pour leurs billets.

**Comment la survie dépendait-elle de l'âge des passagers? Vérifiez (graphiquement) l'hypothèse selon laquelle les jeunes (<30 ans) survivaient plus fréquemment que les personnes âgées (> 55 ans).**

Construisons d'abord la boîte à moustaches (`boxplot`)

In [ ]:
# ...

Nous pouvons visualiser mieux en ajoutant de la couleur aux classes (`hue`).

In [ ]:
# ...

Difficile de conclure quoi que ce soit. Faisons-le d'une autre manière en catégorisant par âge et en utilisant un diagramme en bâtons.

In [ ]:
# ...